In [1]:
import pandas as pd

In [44]:
df = pd.read_csv('tweets_5000.csv')
df.head()

,user,created_at,text
0,wingdongi,10/22/2017 14:22,b'@washingtoen Kaya lu'
1,nujateng,10/22/2017 14:21,b'Gebyar Budaya Hari Santri Nasional 2017 di K...
2,desupriyadi_,10/22/2017 14:21,"b'@detikcom Jujur, bergantung pada paketan dat..."
3,sq_stony,10/22/2017 14:21,b'@IHateJDollarz What'
4,fahmimuttaqiin,10/22/2017 14:21,b'@Alvvin29 @IndosatCare Hahahahaha.. Bener ba...


In [40]:
def remove_b(row):
    return row[2:-1]

In [49]:
df['normalize'] = df['text'].apply(remove_b)
df.head()

,user,created_at,text,normalize
0,wingdongi,10/22/2017 14:22,b'@washingtoen Kaya lu',@washingtoen Kaya lu
1,nujateng,10/22/2017 14:21,b'Gebyar Budaya Hari Santri Nasional 2017 di K...,Gebyar Budaya Hari Santri Nasional 2017 di Kab...
2,desupriyadi_,10/22/2017 14:21,"b'@detikcom Jujur, bergantung pada paketan dat...","@detikcom Jujur, bergantung pada paketan data ..."
3,sq_stony,10/22/2017 14:21,b'@IHateJDollarz What',@IHateJDollarz What
4,fahmimuttaqiin,10/22/2017 14:21,b'@Alvvin29 @IndosatCare Hahahahaha.. Bener ba...,@Alvvin29 @IndosatCare Hahahahaha.. Bener banget


In [54]:
import re

In [164]:
# remove unicode emoji
def remove_emoji(row):
    p = re.compile(r'\\\w\w\d|\\[xf0]\w[^\d]', re.IGNORECASE)
    return p.sub("", row)

In [166]:
print(df['normalize'][9])

RT @dns_dini: #HappyBirthday\xe9\x9e\xa0\xe5\xa9\xa7\xe7\xa5\x8e0618 #kiku semoga bisa nomer 1 di ssk tahun ini amin \nwish you all the best https://t.co/dslKXQeueE


In [167]:
df['normalize'] = df['normalize'].apply(remove_emoji)
print(df['normalize'][9])

RT @dns_dini: #HappyBirthday0618 #kiku semoga bisa nomer 1 di ssk tahun ini amin \nwish you all the best https://t.co/dslKXQeueE


In [182]:
# remove \n
def remove_n(row):
    return re.sub(r"(\\\w(?=[^\w]))|\\\w(?=\w\w\w\w[^_][^:])|(?<=! \\\w)\\\w", " ", row)

In [183]:
df['normalize'] = df['normalize'].apply(remove_n)
print(df['normalize'][141])

Semester 6   simulasi diskusi panel  #moderator #discussion #panel #seminar #nasional #jurusan https://t.co/oMaH1gY8oS


In [187]:
import html

def html_parser(row):
    return html.unescape(row)

In [188]:
df['normalize'] = df['normalize'].apply(html_parser)
print(df['normalize'][141])

Semester 6   simulasi diskusi panel  #moderator #discussion #panel #seminar #nasional #jurusan https://t.co/oMaH1gY8oS


In [189]:
import itertools

def standardizing_words(row):
    return ''.join(''.join(s)[:2] for _, s in itertools.groupby(row))

In [191]:
df['normalize'] = df['normalize'].apply(standardizing_words)
print(df['normalize'][141])

Semester 6  simulasi diskusi panel  #moderator #discussion #panel #seminar #nasional #jurusan https://t.co/oMaH1gY8oS


In [192]:
def clean_url(row):
    return re.sub(r"http\S+", "", row)

In [193]:
df['normalize'] = df['normalize'].apply(clean_url)
print(df['normalize'][141])

Semester 6  simulasi diskusi panel  #moderator #discussion #panel #seminar #nasional #jurusan 


In [194]:
from modulenorm.modNormalize import normalize
from modulenorm.modTokenizing import tokenize

In [220]:
idx = 0
for row in df['normalize']:
    
    # normalize
    usenorm = normalize()
    text_norm = usenorm.enterNormalize(row) # normalisasi enter, 1 revw 1 baris
    text_norm = usenorm.lowerNormalize(text_norm) # normalisasi huruf besar ke kecil
    text_norm = usenorm.repeatcharNormalize(text_norm) # normalisasi titik yang berulang
    text_norm = usenorm.linkNormalize(text_norm) # normalisasi link dalam text
    text_norm = usenorm.spacecharNormalize(text_norm) # normalisasi spasi karakter
    text_norm = usenorm.ellipsisNormalize(text_norm) # normalisasi elepsis (…)
    
    # tokenize
    tok = tokenize() 
    text_norm = tok.WordTokenize(text_norm) # pisah tiap kata pada kalimat

    text_norm = usenorm.spellNormalize(text_norm) # cek spell dari kata perkata
#     text_norm = usenorm.wordcNormalize(text_norm,2) # menyambung kata (malam-malam) (param: textlist, jmlh_loop)
    # text_norm = usenorm.stemmingNormalize(text_norm,'word') # mengubah ke bentuk kata dasar (text, type_data)

    text_norm = ' '.join(text_norm) # menggabung kalimat tokenize dengan separate spasi

    text_norm = usenorm.emoticonNormalize(text_norm) # menggabung struktur emoticon yang terpisah ([: - )] = [:-)])

    # walking2
    # konfer @ ke at untuk penunjuk tempat
    
#     output = open("output1.txt","a")
#     output.write(str(text_norm))
#     output.write('\n')
#     output.close()

#     print(no, text_norm)
    df['normalize2'][idx] = text_norm
    idx += 1

In [225]:
df.tail()

,user,created_at,text,normalize,normalize2
5104,arviirsya,10/22/2017 7:46,b'@nieaamelya Heuheu',@nieaamelya Heuheu,@ nieaamelya heuheu
5105,nandar_art,10/22/2017 7:46,b'Pas jaman SBY ke semarang untuk meresmikan S...,Pas jaman SBY ke semarang untuk meresmikan SPA...,pas jaman sby ke semarang untuk meresmikan spa...
5106,sharameilyanti,10/22/2017 7:46,b'\xf0\x9f\x94\xa5\xf0\x9f\x94\xa5\xf0\x9f\x94...,,
5107,Yusniiunii,10/22/2017 7:46,b'At Perum Bumi Pucang Gading Semarang [pic] \...,At Perum Bumi Pucang Gading Semarang [pic],at perum bumi pucang gading semarang [ pic ]
5108,BEMUndip_,10/22/2017 7:46,b'#Advokesma\n#JobNCareer\n#CareerTNT https://...,#Advokesma #JobNCareer #CareerTNT,# advokesma # jobncareer # careertnt


In [227]:
df['normalize2'].to_csv('normalize2.csv', header=['tweets'], index=False)